# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `MyNumbers` から **ちょうど1回だけ出現する数（single number）** のうち **最大の数**を 1 行で返す。存在しなければ `NULL` を返す。
* 入力 DF: `MyNumbers(num: int)`
* 出力: `num`（1行1列、single number の最大。なければ `NULL`）

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → 出現回数（`value_counts`）→ 出現1回の要素だけ残す → `max`。`sort_values` は使わず、`LIMIT` 相当の早期終了も不要。

```python
import pandas as pd

def largest_single_number(my_numbers: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        my_numbers (pd.DataFrame): 列 'num' を持つ DataFrame（重複可）
    Returns:
        pd.DataFrame: 列名と順序は ['num']（1行1列）。single number がなければ NULL（pd.NA）を返す。
    """
    # 対象列のみ（列最小化）
    s = my_numbers['num']

    # 各値の出現回数（NaNを数える必要がなければ dropna=True でもよいが、max は NaN を無視するため既定のままでOK）
    vc = s.value_counts(dropna=False)

    # 出現1回である行だけを抽出（セミジョイン相当）
    mask_single = s.map(vc).eq(1)
    candidates = s[mask_single]

    # 最大値を1行で返す。候補が空なら NULL（pd.NA）
    if candidates.empty:
        out = pd.DataFrame({'num': [pd.NA]})
    else:
        out = pd.DataFrame({'num': [candidates.max()]})

    return out

Analyze Complexity
Runtime 298 ms
Beats 33.33%
Memory 67.06 MB
Beats 47.82%

```

## 3) アルゴリズム説明

* 使用 API

  * `Series.value_counts(dropna=False)`: 値ごとの出現回数を計算（ハッシュベース）
  * `Series.map(vc)`: 各行に出現回数を付与（軽量結合）
  * ブールフィルタで `== 1` を抽出
  * `Series.max()`: single 値の中の最大値を取得（`NaN` は既定で無視）
* **NULL / 重複 / 型**

  * `NaN` が1回だけでも `Series.max()` は無視するため、非NULLの single があればそちらが選ばれる（SQL の `MAX` と同等の直感）。
  * 全ての single が存在しない場合（= 候補空）は `pd.NA` を返す。
  * 返却列は1列のみ `num`。`pd.NA` を含み得るため dtype は `Int64` 互換か object になるが、仕様上は列名と値が重要。

## 4) 計算量（概算）

* `value_counts`（ハッシュ集計）: **O(N)**
* `map` による付番＋フィルタ: **O(N)**
* `max`: **O(K)**（K は single の件数）
  合計 **O(N)** 近似、追加メモリは一時ハッシュ（一意値数に比例）。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム]
  B[前処理 列最小化]
  C[出現回数を計算 value_counts]
  D[map で行に回数を付与]
  E[回数が1回の行だけ抽出]
  F[max で最大の数を取得]
  G[出力 列 num だけ]
  A --> B
  B --> C
  C --> D
  D --> E
  E --> F
  F --> G
```

ポイントは **「全行サイズの中間 Series を作らない」** ことと、**余計なソートを避ける**ことです。

---

## 改善ポイント（効果順）

1. **`value_counts(sort=False)` を使ってソートを抑止**
   既定では並べ替えが走ることがありコスト増。`sort=False` で純粋なハッシュ集計にします。

2. **`map`／全行マスクをやめて「ユニーク値側だけ」で最大値を決める**
   `value_counts` の結果（= ユニーク値ごとの回数）から **出現1回のキー**だけ抽出し、その **キー集合の最大**を取れば、元 Series 全体に戻してブールマスクを作る必要がありません。
   → メモリも CPU も削減。

3. （オプション）**超大規模なら NumPy 直利用**
   `np.unique(..., return_counts=True)` で同様のことができます。pandas 生成物を最小化できるので、極端に大きいデータで効くことがあります。

---

## 改良版（純 pandas・シグネチャ厳守）

```python
import pandas as pd

def largest_single_number(my_numbers: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        my_numbers (pd.DataFrame): 列 'num' を持つ DataFrame（重複可）
    Returns:
        pd.DataFrame: 列名と順序は ['num']（1行1列）。single number がなければ NULL（pd.NA）を返す。
    """
    s = my_numbers['num']

    # ユニーク値ごとの出現回数。sort=False で余計な並び替えを抑止
    vc = s.value_counts(dropna=False, sort=False)

    # 出現1回の値だけ（Index）
    singles = vc.index[vc.eq(1)]

    if len(singles) == 0:
        return pd.DataFrame({'num': [pd.NA]})

    # 最大値（NaN/NA は無視）。すべてが NaN のときは NaN が返る → pd.NA に正規化
    max_single = pd.Series(singles).max(skipna=True)
    if pd.isna(max_single):
        return pd.DataFrame({'num': [pd.NA]})

    return pd.DataFrame({'num': [max_single]})
```

**変更点の狙い**

* `map(vc)` と `mask_single`（長さ N の中間 Series）を **作成しない**ため、**メモリ削減**と **CPU 削減**。
* `sort=False` で **集計のみ**に徹して高速化。
* `max(skipna=True)` で **NULL 安全**（SQL の `MAX` 同等の直感）。

---

## さらに攻める版（NumPy 直利用・大規模向け）

```python
import numpy as np
import pandas as pd

def largest_single_number(my_numbers: pd.DataFrame) -> pd.DataFrame:
    s = my_numbers['num']
    vals = s.to_numpy(copy=False)

    # unique と counts を同時取得（unique は dtype 依存でソートされますが影響なし）
    uniq, cnts = np.unique(vals, return_counts=True)

    # 出現1回の候補のみ
    cand = uniq[cnts == 1]

    if cand.size == 0:
        return pd.DataFrame({'num': [pd.NA]})

    # NaN を弾いて最大を取る（数値前提）
    # 数値でない（object）可能性もあるので、pandas 経由で安全に最大を取る
    max_single = pd.Series(cand).max(skipna=True)
    if pd.isna(max_single):
        return pd.DataFrame({'num': [pd.NA]})

    return pd.DataFrame({'num': [max_single]})

Analyze Complexity
Runtime 242 ms
Beats 98.12%
Memory 66.85 MB
Beats 62.31%

```

> 備考: `np.unique` は戻り値をソートしますが、**最大値を取るだけ**なので問題になりません（`sort_values` は未使用）。

---

## なぜ速く・軽くなるか

* 以前の案は `map(vc)`→**長さ N の中間配列**を作っていました。
  改良案は **ユニーク値個数（≪N のことが多い）だけ**を扱い、`max` もそこから計算します。
* `value_counts(sort=False)` で **集計のみ**に限定し、不要な並び替えコストを排除。

---

## 概算計算量とメモリ

* 集計: **O(N)**（ハッシュ）
* 以降は **O(U)**（U=ユニーク値数）
* 追加メモリ: ハッシュテーブル（U に比例）＋小さな中間（Series(singles)）

---

## 仕上げの小ネタ

* `my_numbers['num']` の **dtype を整数系（nullable Int64）や適切な数値型に揃える**と、`value_counts` の内部ハッシュが効きやすいケースがあります。
* 事前に `my_numbers = my_numbers[['num']]` と **列最小化**してから渡す（既に満たしていれば不要）。

---

これで **メモリ 1 枚分の中間 Series を削り**つつ、**余計な並べ替えをカット**できるので、`Runtime`/`Memory` ともに改善が見込めます。

